<a href="https://colab.research.google.com/github/opendatadurban/City-Resilience-Tool/blob/main/ops_toggl_analysis_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Operations Toggl Data Analysis

## Inital set up

In [1]:
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default
from googleapiclient.discovery import build
import math
import io
import numpy as np

from google.colab import drive

#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
# Get the Google Drive API service
drive_service = build('drive', 'v3', credentials=creds)

drive.mount('/content/drive')

data_path = '/content/drive/Shareddrives/OCL | Office Space/Domains/Operations/Ops_Toggl_Tracking/data'
out_path = '/content/drive/Shareddrives/OCL | Office Space/Domains/Operations/Ops_Toggl_Tracking/outputs'

Mounted at /content/drive


## Get forecast data

Get data from [OCL Team Capacity Tracker (2024)](https://docs.google.com/spreadsheets/d/1scCmK-KPr4DchZ0ych68iwHPwMH1j9ruhltYbXP6Yrg/edit?gid=0#gid=0)


Here we want to get the number of expected days for each person PER MONTH from the original planning document. As long as the structure of the document hasn't changed, this should work and should pull in any updates to the assigned days. If rows or columns have been added or removed, this script will need to be edited to account for that.

**NB!!!! SELECT THE CORRECT SHEET NAME**

In [9]:
# Get the expected number of days per month per person per project
wb = 'OCL Team Capacity Tracker (2024)'

worksheet = gc.open(wb)

# Select the specific sheet by name
sheet_name = 'Overview'

sheet = worksheet.worksheet(sheet_name)
# Get all the values from the sheet
all_values = sheet.get_all_values()
# Convert the data to a DataFrame
# Use the first row as the header and the rest as data

df = pd.DataFrame(all_values[3:], columns=all_values[0])
df

,Team Member,Role,Available Days/Month,OCL Admin+Internal,FCDO (forecast),Gates (forecast),Constititionalism Fund (forecast),EDGE (forecast),SA - IDL (forecast),Service Delivery Simulator (DCoG),...,EDGE (actual),SA - IDL (actual),Service Delivery Simulator (actual),GWOPA (actual),Impact & Advocacy (actual),SMT (actual),Uganda (actual),Capacity remaining (actual),DIFFERENCE,@dropdown
0,Aliasgher,Product Manager,17,3.4,12.17,,,,,0.50,...,,,,,,,,,,
1,Anele,Data Scientist,17,3.4,1.00,,,,,,...,,,,,,,,,,
2,Ayanda M,Product Manager,17,3.4,7.00,12.00,,,,,...,,,,,,,,,,
3,Ayanda N,Dev FE,17,3.4,4.83,2.00,1.00,1.92,,6.33,...,,,,,,,,,,
4,Benjine,UX Designer,17,3.4,2.00,4.00,0.25,,,5.00,...,,,,,,,,,,
5,Brian,Data Scientist,11.5,2.3,0,2.00,0.44,,0.24,0.50,...,,,,,,,,,,
6,Ella,Product Manager,17,3.4,4.67,,4.56,,,,...,,,,,,,,,,
7,Heiko,Data Scientist,17,1.7,9.00,4.00,,,4.47,,...,,,,,,,,,,
8,Lerato H,Comms coord.,,,,2.00,0.25,,,,...,,,,,,,,,,
9,Lerato M,Designer,17,3.4,4.33,4.00,,1.92,,2.67,...,,,,,,,,,,


## Get Toggl data

This is the data that you have downloaded from Toggl, uploaded [here](https://drive.google.com/drive/folders/1phmgvZ65VqqOZEQ9ZNZRpXMIZCIOHCfn) and renamed **data**. It is important that this data is in csv format, grouped by User and Task.


In [10]:
df_data = pd.read_csv(f'{data_path}/data.csv')
# convert Duration to a time unit
df_data = df_data.loc[:,['User','Project','Task','Duration']]
df_data


## Next up

### Step 1
Manually create lookup tables to match project, task and people names in toggl data to project, task and people names in forecast data. These tables should sit in the [data](https://drive.google.com/drive/folders/1phmgvZ65VqqOZEQ9ZNZRpXMIZCIOHCfn) folder. Whether you make each table a separate sheet in a single document or 3 separate documents is up to you.

### Step 2
Use these lookup tables to change the names in the toggl data to match the forecast data

### Step 3
For each user, group the data by project, convert to days and round to the nearest 0.25 (NB CONFIRM what this value should be with standard Ops protocol).
Save raw data and processed data in [outputs](https://drive.google.com/drive/folders/1BYLyBpM6yCEZ37rGfl8XTfl6dtayVMcS).

### Step 4
Combine processed data with forecast data as expected in the [OCL Team Capacity Tracker (2024)](https://docs.google.com/spreadsheets/d/1scCmK-KPr4DchZ0ych68iwHPwMH1j9ruhltYbXP6Yrg/edit?gid=0#gid=0) overview page and save as a new file in the [working directory](https://drive.google.com/drive/folders/1Aq4Bv7h_Pk-slt7Q8SjkAH8JopAr_IRQ) for now. Once we are happy that this is working as expected, we can look to update the existing OCL Team Capacity Tracker.
